In [4]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import PowerTransformer
import math
import warnings
warnings.filterwarnings('ignore')

# 處理 data 套件
import numpy as np
import pandas as pd
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,KFold
from scipy.stats import skew
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from mlxtend.regressor import StackingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.svm import SVR

In [5]:
from imbdpack import skew_pro,lr_rmse_ave,lr_rmse_ave_fea,laso_rmse_ave,ElasticNet_rmse_ave,xgb_ave,svr_rmse_ave,rand_ave
from imbdpack import voting_ave,stack_ave,ave

In [6]:
fea_number=14
data=pd.read_csv('../BayesianRidge_Pre_0/Input_A1_020.csv',index_col=0).drop(['Number'], axis=1)
y=data.Predict
skew_data=skew_pro(data)
num=skew_data.select_dtypes(exclude='object')
numcorr=num.corr()
title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea_number+1).to_frame().index.to_numpy()[1:]
title

array(['Input_A4_020', 'Input_A3_020', 'Input_A5_020', 'Input_A6_020',
       'Input_A2_020', 'Input_A1_021', 'Input_A2_021', 'Input_A2_015',
       'Input_C_001', 'Input_A1_016', 'Input_C_060', 'Input_A1_019',
       'Input_A4_011', 'Input_A2_019'], dtype=object)

## Linear Regression Baseline

In [7]:
lr_rmse_ave_fea(skew_data,14)

train_rmse: 0.5496703758835424
test_rmse: 0.630685153741493
test_r2: 0.40211639004494143


## Lasso

In [8]:
laso_rmse_ave(skew_data[title],y,0.01)

test_rmse_ave: 0.5966543862816749
[0.46620984911224955, 0.6741264570147933, 0.8203464865214706, 0.4369860546979481, 0.6401251047894848, 0.7456655545103595, 0.7047084919188028, 0.6229993612177261, 0.4866207589069143, 0.36875574412700046]


test_r2_ave: 0.4172470972694066
[0.5921123062469382, 0.33779988139965444, 0.41364601517656396, 0.4194992900160419, 0.5331287816228147, 0.23563419886713255, 0.32234871859884107, 0.3085633111767526, 0.4348570496571701, 0.5748814199321561]


## ElasticNet

In [9]:
ElasticNet_rmse_ave(skew_data[title],y,[0.09],0.02)

test_rmse_ave: 0.5899816492677211
[0.471493396911073, 0.6503581355816648, 0.8399015358747844, 0.4255020295801734, 0.643091779783245, 0.7324902145547428, 0.714958298947838, 0.5941570359776989, 0.478868904597491, 0.34899516086850063]


test_r2_ave: 0.4338190480252001
[0.5828147508562611, 0.3836723310827619, 0.38535834861637475, 0.4496095730017958, 0.5287913022957005, 0.2624070804541806, 0.30249282821691914, 0.3711027346564324, 0.4527190577421931, 0.6192224733293814]


## SVR

In [10]:
model=SVR(kernel='linear', C=1.3, gamma= 1e-07, epsilon= 0.4)
svr_rmse_ave(skew_data,14,model)

train_rmse: 0.5648260361049923
test_rmse: 0.6284314864841181
test_r2: 0.409871488238135


## XGB

In [11]:
best_xgb_model = XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.09,
              booster = 'gbtree', 
              n_estimators = 30, 
              max_depth = 5, 
              min_child_weight = 4,
              seed = 42,
              gamma = 0.2,
              subsample = 0.4,
              colsample_bytree = 0.5,
              reg_alpha =  0.5,
              reg_lambda = 1.5)
xgb_ave(skew_data[title], y,best_xgb_model)

test_rmse_ave: 0.5569718878885064
[0.4334438552531182, 0.5786805474292152, 0.8481278502741105, 0.38215855005682964, 0.6337833355740593, 0.641218727362057, 0.6814416241759549, 0.5845952968696175, 0.3994497373408201, 0.38681935454928223]


test_r2_ave: 0.4974809869095451
[0.6474315752063531, 0.5120399382958085, 0.37325931892128694, 0.5560288388691279, 0.5423336170906468, 0.4347696408763295, 0.3663571045757962, 0.39118148466533287, 0.6191961496004228, 0.5322122009943471]


## RandomForest

In [12]:

rand_model = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 500,
            max_depth = 2,
            min_samples_split = 7,
            max_leaf_nodes = 3,
            min_samples_leaf =5,
            random_state = 42) 
rand_ave(skew_data[title], y,rand_model)

test_rmse_ave: 0.5992695000065618
[0.42449305712875873, 0.6765634229439021, 0.8571850322298274, 0.46536364368303335, 0.6550714383674187, 0.6951032756058838, 0.7247560260997505, 0.6925960420160501, 0.4417597571020723, 0.359803304888921]


test_r2_ave: 0.41013784632330685
[0.6618425988269516, 0.3330035231537237, 0.35980188048584527, 0.3416565756421902, 0.5110722241981486, 0.3357802960066595, 0.28324468147231285, 0.14545049250183517, 0.5342537657333033, 0.5952724252120978]


In [13]:
lasso_mod=Lasso(alpha=0.0009)
elastic_mod=ElasticNet(alpha=[0.09], l1_ratio=0.02)
svr_mod=SVR(kernel='linear', C=1.3, gamma= 1e-07, epsilon= 0.4)

xgb_mod = XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.09,
              booster = 'gbtree', 
              n_estimators = 30, 
              max_depth = 5, 
              min_child_weight = 4,
              seed = 42,
              gamma = 0.2,
              subsample = 0.4,
              colsample_bytree = 0.5,
              reg_alpha =  0.5,
              reg_lambda = 1.5)

random_mod = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 500,
            max_depth = 2,
            min_samples_split = 7,
            max_leaf_nodes = 3,
            min_samples_leaf =5,
            random_state = 42) 
vote_mod = VotingRegressor([ ('Lasso', lasso_mod), ('Elastic', elastic_mod), 
                            ('XGBRegressor', xgb_mod),('RandomForest', random_mod)])

voting_ave(skew_data[title], y,vote_mod)


test_rmse_ave: 0.5697043721380013
[0.4286501110237225, 0.6291318978863927, 0.8294365970140347, 0.40625206292070454, 0.6278240599796704, 0.6909499119036118, 0.6968223198119645, 0.6083275641902436, 0.43798724801766664, 0.34166194863200217]


test_r2_ave: 0.4727298865875322
[0.6551870283493997, 0.4232469226000336, 0.400579455853711, 0.4982830675388479, 0.5508997553181383, 0.34369423947480204, 0.33743062199981777, 0.3407468579809859, 0.5421744945122011, 0.6350564222473842]


In [14]:

stack_mod = StackingRegressor(regressors=[lasso_mod, elastic_mod, random_mod, vote_mod], 
                           meta_regressor=xgb_mod, use_features_in_secondary=True
                          )

stack_ave(skew_data[title], y,stack_mod)

test_rmse_ave: 0.5665450423066548
[0.4035687013255988, 0.6001928610971173, 0.7755356746403117, 0.4272796814422853, 0.597367544278411, 0.6941806008946317, 0.6943679906587507, 0.7092528708162772, 0.3894267232899385, 0.37427777462322614]


test_r2_ave: 0.4667421413549443
[0.6943582562320338, 0.47508599368676374, 0.47595474079876876, 0.4450011515498893, 0.5934156782512514, 0.337542485865689, 0.342089771052698, 0.10385265550950273, 0.6380666921853011, 0.5620539884175444]


In [15]:
ave(skew_data[title],y,0.5,stack_mod,0.3,vote_mod,0.2,xgb_mod)

test_rmse_ave: 0.5572564093737785
[0.40489901896494174, 0.5964709411250032, 0.8017642638241619, 0.4012638218375119, 0.6073963382910789, 0.6749895752925594, 0.6883024702795874, 0.6423545810458476, 0.39967377117445024, 0.3554493119026423]


test_r2_ave: 0.4919910781865039
[0.6923399096273888, 0.48157600860926686, 0.43990895866915203, 0.5105282739083397, 0.5796493528133431, 0.3736642289998353, 0.35353366976016953, 0.2649330966686939, 0.6187688774676197, 0.6050084053412307]


In [16]:
ave(skew_data[title],y,0.5,stack_mod,0.2,vote_mod,0.3,xgb_mod)

test_rmse_ave: 0.5561249151803935
[0.40511152298313813, 0.5911779107396677, 0.8039008364289766, 0.3996572184821266, 0.6077795401113436, 0.6701689511231701, 0.6869690851310399, 0.640259528389218, 0.3956729716757026, 0.3605515867395523]


test_r2_ave: 0.49415165702376884
[0.6920168850761728, 0.49073608194496576, 0.43691987646018104, 0.5144399778000259, 0.5791187933258866, 0.3825785819895672, 0.35603592339223267, 0.2697201489817671, 0.6263630479971938, 0.5935872532696951]


In [17]:
ave(skew_data[title],y,0.35,stack_mod,0.15,vote_mod,0.5,xgb_mod) ###!!!!

test_rmse_ave: 0.5544394216013309
[0.41010257546031437, 0.5855111226538348, 0.8161217261677147, 0.3926346076323206, 0.6135996562681292, 0.6601350493801293, 0.6845670392119708, 0.6210235869207151, 0.3951777222371947, 0.3655211300809855]


test_r2_ave: 0.49908837045461807
[0.6843813138578994, 0.5004524775377999, 0.41966987544099843, 0.5313541745555943, 0.5710194547842253, 0.4009284872163442, 0.36053140070554746, 0.31294198848576715, 0.6272977981460619, 0.5823067338159433]


In [18]:
ave(skew_data[title],y,0.3,stack_mod,0.1,vote_mod,0.6,xgb_mod)

test_rmse_ave: 0.5541699433733891
[0.41308111590261315, 0.5826482391127873, 0.8211947134560721, 0.39032564534872455, 0.6163576929518495, 0.6557282850321062, 0.6836036756623655, 0.6145836624002959, 0.3944719647074227, 0.36970443915965445]


test_r2_ave: 0.5001209322451847
[0.6797800411551549, 0.5053256556606625, 0.41243282444888596, 0.5368498889410571, 0.5671543838243263, 0.40890005416005415, 0.36232993098757316, 0.32711749031496595, 0.628627845302532, 0.5726912076566357]
